In [9]:
import pandas as pd
import numpy as np
import math
data = pd.read_csv("mutationc.csv",index_col=0)
np.random.seed(52)

### Phi Computation

In [10]:
idx = data.index.astype(str)
C_mask = data.index.to_series().str.match(r"^C\d+$")
NC_mask = data.index.to_series().str.match(r"^NC\d+$")

def entropy(P_C_t,P_NC_t):
    ans = 0.0
    if P_C_t > 0:
        ans -= P_C_t * math.log2(P_C_t)
    if P_NC_t > 0:
        ans -= P_NC_t * math.log2(P_NC_t)
    return ans

def information_gain(group:pd.DataFrame):
    n = group.shape[0]
    is_C = C_mask.loc[group.index]
    is_NC = NC_mask.loc[group.index]
    # calc that can be done for group
    n_t_C = int(is_C.sum())
    n_t_NC = int(is_NC.sum())
    P_C_t = n_t_C / n
    P_NC_t = n_t_NC / n
    H_t = entropy(P_C_t, P_NC_t)
    rows = []
    for feature in group.columns:
        col = group[feature]
        left_mask = (col == 1)
        right_mask = ~left_mask
        n_tL = int(left_mask.sum())
        n_tR = n - n_tL
        n_tL_C  = int((left_mask  & is_C).sum())
        n_tL_NC = int((left_mask  & is_NC).sum())
        n_tR_C  = int((right_mask & is_C).sum())
        n_tR_NC = int((right_mask & is_NC).sum())
        P_L = n_tL / n
        P_R = n_tR / n
        P_C_tL  = n_tL_C  / n_tL if n_tL != 0 else 0
        P_NC_tL = n_tL_NC / n_tL if n_tL != 0 else 0
        P_C_tR  = n_tR_C  / n_tR if n_tR != 0 else 0
        P_NC_tR = n_tR_NC / n_tR if n_tR != 0 else 0
        H_st = P_L * entropy(P_C_tL, P_NC_tL) + P_R * entropy(P_C_tR, P_NC_tR)
        gain = H_t - H_st
        rows.append((feature, gain))
        
    return pd.DataFrame(rows, columns=["feature", "gain(s)"]).set_index("feature")


### Random Forest Generation

In [11]:
class tree:
    def __init__(self,F=None,alpha=None,beta=None,leaves=None,oob=None):
        self.F = F
        self.alpha = alpha
        self.beta = beta
        self.leaves = leaves
        self.oob = oob

    def classify(self,sample):
        if sample[self.F] == 1:
            if sample[self.alpha] == 1:
                return self.leaves["A1"]
            else:
                return self.leaves["A2"]
        else:
            if sample[self.beta] == 1:
                return self.leaves["B1"]
            else:
                return self.leaves["B2"]
            
def leaf_majority(leaf:pd.DataFrame):
    leaf_idx = leaf.index.to_series().replace(r'^C\d+$','1',regex=True).replace(r'^NC\d+$','0',regex=True).astype(int)
    if leaf_idx.mean() > 0.5:
        return 1 # majority cancer
    else:
        return 0 # majority noncancer

def create_tree(group:pd.DataFrame):
    bootstrap = group.sample(n=len(group),replace=True)
    oob = group.loc[~group.index.isin(bootstrap.index)]
    k = math.ceil(math.sqrt(group.shape[1]))
    F = information_gain(bootstrap).sample(n=k)["gain(s)"].idxmax()

    groupA = bootstrap[bootstrap[F] == 1]
    groupB = bootstrap[bootstrap[F] == 0]

    alpha = information_gain(groupA).sample(n=k)["gain(s)"].idxmax()
    beta = information_gain(groupB).sample(n=k)["gain(s)"].idxmax()

    ln_a1  = groupA[groupA[alpha] == 1]
    ln_a2 = (groupA[groupA[alpha] == 0])
    if len(ln_a1) == 0: ln_a1 = groupA
    if len(ln_a2) == 0: ln_a2 = groupA

    ln_b1 = groupB[groupB[beta] == 1]
    ln_b2 = groupB[groupB[beta] == 0]
    if len(ln_b1) == 0: ln_b1 = groupB
    if len(ln_b2) == 0: ln_b2 = groupB

    leaves = {"A1":leaf_majority(ln_a1),"A2":leaf_majority(ln_a2),
              "B1":leaf_majority(ln_b1),"B2":leaf_majority(ln_b2)}

    return tree(F,alpha,beta,leaves,oob)

In [12]:
print("Forest Generation")
NUM_TREES = 25
forest = []
oob_sum = 0.0
for trial in range(1,NUM_TREES+1):
    gen_tree = create_tree(data)
    print(f" Tree {trial} --\n\tRoot:{gen_tree.F}\n\tAlpha:{gen_tree.alpha}\n\tBeta:{gen_tree.beta}\n\tOOB Size:{len(gen_tree.oob)}\n")
    oob_sum += len(gen_tree.oob)
    forest.append(gen_tree)
print(f"OOB Average: {oob_sum / NUM_TREES}")

def forest_classifier(idx,sample,forest):
    votes = []
    for t in forest:
        votes.append(t.classify(sample))
    votes_for_C = sum(votes)
    votes_for_NC = len(votes) - votes_for_C
    if votes_for_C > votes_for_NC:
        label = "C"
        value = 1
    else:
        label = "NC"
        value = 0
    return pd.Series({"idx":idx,"label":label, "value":value,
            "C_votes":votes_for_C,"NC_Votes":votes_for_NC})

Forest Generation
 Tree 1 --
	Root:ARID1A_GRCh37_1:27105931-27105931_Frame-Shift-Del_DEL_G-G--
	Alpha:SLC27A2_GRCh37_15:50515214-50515214_Missense-Mutation_SNP_G-G-A
	Beta:CCDC39_GRCh37_3:180332568-180332568_3'UTR_DEL_T-T--
	OOB Size:73

 Tree 2 --
	Root:PHACTR1_GRCh37_6:13206135-13206135_Frame-Shift-Del_DEL_G-G--
	Alpha:ZNF142_GRCh37_2:219508084-219508084_Frame-Shift-Del_DEL_C-C--
	Beta:KRAS_GRCh37_12:25380275-25380275_Missense-Mutation_SNP_T-T-G
	OOB Size:79

 Tree 3 --
	Root:PCDH19_GRCh37_X:99662505-99662505_Frame-Shift-Del_DEL_G-G--
	Alpha:APC_GRCh37_5:112151204-112151204_Nonsense-Mutation_SNP_C-C-T
	Beta:RAB28_GRCh37_4:13485808-13485808_5'UTR_DEL_G-G--
	OOB Size:77

 Tree 4 --
	Root:CNTNAP5_GRCh37_2:125672808-125672808_3'UTR_SNP_C-C-T
	Alpha:PLCB3_GRCh37_11:64032517-64032517_Frame-Shift-Del_DEL_C-C--
	Beta:SALL1_GRCh37_16:51172618-51172618_Missense-Mutation_SNP_G-G-A
	OOB Size:75

 Tree 5 --
	Root:OR6K2_GRCh37_1:158669532-158669532_Frame-Shift-Del_DEL_T-T--
	Alpha:GJA4_GRCh37_1:35

## Classifying Set of Samples

In [13]:
samples = ["C1","C10","C15","NC5","NC15"]
for sample in samples:
    result = forest_classifier(sample,data.loc[sample],forest)
    print(sample,"-> predicted",result.label)


C1 -> predicted C
C10 -> predicted C
C15 -> predicted C
NC5 -> predicted NC
NC15 -> predicted NC


## Classifying Out-Of-Bag Samples

In [14]:
samples = forest[0].oob.index.to_list()
# print("OOB Classifications--")
oob = forest[0].oob
oob_classification = []
for sample in samples:
    result = forest_classifier(sample,oob.loc[sample],forest)
    # print(sample,"-> predicted ",result.label)
    oob_classification.append(result)

In [15]:
predicted = pd.Series([s.value for s in oob_classification],index=samples)
actual = oob.index.to_series().replace(r'^C\d+$','1',regex=True).replace(r'^NC\d+$','0',regex=True).astype(int)

In [16]:
def compute_metrics(actual:pd.Series,predicted:pd.Series):
    TP = ((actual == 1) & (predicted == 1)).sum(axis=0)
    FP = ((actual == 0) & (predicted == 1)).sum(axis=0)
    FN = ((actual == 1) & (predicted == 0)).sum(axis=0)
    TN = ((actual == 0) & (predicted == 0)).sum(axis=0)
    Accuracy = (TP + TN) / (TP + TN + FP + FN)
    Sensitivity = TP / (TP + FN)
    Specificity = TN / (TN + FP)
    Precision = TP / (TP + FP)
    Miss_rate = FN / (TP + FN)
    False_discovery_rate = FP / (TP + FP)
    False_omission_rate = FN / (FN + TN)
    metrics = pd.DataFrame({"Accuracy":Accuracy,"Sensitivity":Sensitivity,
                            "Specificity":Specificity,"Precision":Precision,
                            "Miss Rate":Miss_rate,"False Discovery Rate":False_discovery_rate,
                            "False Omission Rate":False_omission_rate},index=[0])
    return metrics
results = compute_metrics(actual,predicted)

In [17]:
results

,Accuracy,Sensitivity,Specificity,Precision,Miss Rate,False Discovery Rate,False Omission Rate
0,0.643836,0.351351,0.944444,0.866667,0.648649,0.133333,0.413793
